# CW1 - Metacritic Analysis with PySpark

In this assignment I will be  exploring a dataset containing Metacritic entries up to year 2020.

In [ ]:
# CodeGrade Tag Init1

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# CodeGrade Tag Init2

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Unpack Spark from google drive
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "spark-3.3.0-bin-hadoop3"

# Install findspark, which helps python locate the psyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [ ]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

# pyspark.sql.functions countains all the transformations and actions you will
# need

from pyspark.sql import functions as F
from pyspark.sql.functions import col

In [ ]:
# Load the games-data.csv file into your notebook as a pyspark dataframe

CsvPath = '/content/games-data.csv'

# Load .csv with header, ',' seperators and inferred schema
GamesDF = spark.read\
                     .option('header', 'True')\
                     .option("escapeQuotes", "True")\
                     .option('sep', ',')\
                     .option('inferSchema', 'True')\
                     .csv(CsvPath)

In [ ]:
# In this cell we perform some minimal preprocessing
# First, drop all rows containing null values

GamesDF = GamesDF.na.drop()

# Second, we extract the year from the release date

GamesDF = GamesDF.withColumn("year",
                             F.regexp_extract("r-date", r"(.),(\s+)(\w+)", 3))
GamesDF = GamesDF.withColumn('year', GamesDF['year'].cast('int'))
GamesDF = GamesDF.drop('r-date')

In [ ]:
# CodeGrade Tag Init3

GamesDF.printSchema()
GamesDF.show()

root
 |-- name: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- user score: string (nullable = true)
 |-- developer: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- players: string (nullable = true)
 |-- critics: integer (nullable = true)
 |-- users: integer (nullable = true)
 |-- year: integer (nullable = true)

+--------------------+------------+-----+----------+--------------------+--------------------+--------------------+-------+-----+----+
|                name|    platform|score|user score|           developer|               genre|             players|critics|users|year|
+--------------------+------------+-----+----------+--------------------+--------------------+--------------------+-------+-----+----+
|The Legend of Zel...|  Nintendo64|   99|       9.1|            Nintendo|Action Adventure,...|            1 Player|     22| 5749|1998|
|Tony Hawk's Pro S...| PlayStation|   98|       7.4|NeversoftEnter

Filter and show all the titles with a score higher or equal to 95 released from 2017

In [ ]:
# CodeGrade Tag Ex1a
### Filter and show all the titles with a score higher or equal to 95
### released from 2017

GamesDF.filter((GamesDF['year']>=2017)&(GamesDF['score']>=95)).show(truncate=False)

+---------------------------------------+------------+-----+----------+-------------+-------------------------------------------+---------------------+-------+-----+----+
|name                                   |platform    |score|user score|developer    |genre                                      |players              |critics|users|year|
+---------------------------------------+------------+-----+----------+-------------+-------------------------------------------+---------------------+-------+-----+----+
|Red Dead Redemption 2                  |XboxOne     |97   |7.8       |RockstarGames|Action Adventure,Open-World                |Up to 32             |33     |3186 |2018|
|The Legend of Zelda: Breath of the Wild|Switch      |97   |8.6       |Nintendo     |Action Adventure,Open-World                |No Online Multiplayer|109    |15873|2017|
|Red Dead Redemption 2                  |PlayStation4|97   |8.4       |RockstarGames|Action Adventure,Open-World                |Up to 32        

Create a dataframe that contains, for each developer, the number of distinct titles, the average score, the average user score, the total number of critics and the total number of users.

Make sure the columns have the following names:

```developer|Number of Titles|Average Score|Average User Score|Total Critics|Total Users```

Sort by the number of titles in descending order.

In [ ]:
from pyspark.sql import functions as F

In [ ]:
# CodeGrade Tag Ex1b
### Create a dataframe that contains, for each developer, the number of distinct
### titles, the average score, the average user score, the total number of
### critics and the total number of users. Sort by the number of titles in
### descending order.

GamesDF.groupBy('developer').agg(F.count_distinct('name').alias('Number of Titles'),F.avg('score').alias('Average Score'),F.avg('user score').alias('Average User Score'),F.sum('critics').alias('Total Critics'),F.sum('users').alias('Total Users')).sort('Number of titles',ascending=[False]).show()

+------------------+----------------+-----------------+------------------+-------------+-----------+
|         developer|Number of Titles|    Average Score|Average User Score|Total Critics|Total Users|
+------------------+----------------+-----------------+------------------+-------------+-----------+
|            Capcom|             184| 74.9054054054054| 7.848620689655175|         8463|      58471|
|          Nintendo|             140|80.38028169014085| 7.963309352517986|         5926|      82108|
|     TelltaleGames|             120|75.30932203389831| 7.151709401709399|         4227|      26054|
|            Konami|             120|69.84180790960453| 6.998726114649679|         3470|      13437|
|           Ubisoft|              90|67.67647058823529|6.8560606060606055|         3031|      24671|
|          EASports|              79| 79.8051282051282|  6.60374331550802|         4294|      20058|
|        SquareEnix|              78|75.38317757009345| 7.583168316831683|         3580|   

Print the total number of multi-platform titles in the data, i.e. the distinct titles which appear for *at least two* different platforms.

In [ ]:
# CodeGrade Tag Ex2a
### Print the total number of multi-platform titles in the data, i.e.
### the distinct titles which appear for at least two different platforms

n_multi_titles=GamesDF.groupby('name').agg(F.count_distinct('platform')).where('count(platform) >= 2').select('name').count()

print(f"Number of multi-platform titles: {n_multi_titles}")

Number of multi-platform titles: 3615


Print the name of the developer which has developed titles for the highest number of distinct platforms.

In [ ]:
# CodeGrade Tag Ex2b
### Print the developer which has developed titles for the highest number
### of distinct platforms

developer_most_platforms = GamesDF.groupby('developer').agg(F.count_distinct('platform').alias('platform')).sort('platform',ascending=(False)).first()[0]

print(f"Developer for highest number of platforms: {developer_most_platforms}")

Developer for highest number of platforms: Konami


Group the data by platform. Show, for each platform, its highest rated title, the total number of titles and the total number of critics.

Create a dataframe where the columns are:

```Platform|Highest Rated Tile|Number of Titles|Number of Critics```

Sort by the Number of Titles in descending order.

In [ ]:
# CodeGrade Tag Ex3
### Group the data by platform. Show, for each platform, its highest rated
### title, the total number of titles and the total number of critics.
### Sort by the Number of Titles in descending order.

GamesDF.sort('score',ascending=(False)).groupBy('Platform').agg(F.first('name').alias('Highest Rated Title'),F.count('name').alias('Number of Titles'),F.sum('critics').alias('Number of Critics')).sort('Number of Titles',ascending=(False)).show(truncate=False)

+---------------+---------------------------------------+----------------+-----------------+
|Platform       |Highest Rated Title                    |Number of Titles|Number of Critics|
+---------------+---------------------------------------+----------------+-----------------+
|PC             |Half-Life 2                            |4578            |89340            |
|PlayStation4   |Red Dead Redemption 2                  |1926            |51104            |
|Xbox360        |Grand Theft Auto IV                    |1665            |57283            |
|PlayStation2   |Tony Hawk's Pro Skater 3               |1418            |34278            |
|PlayStation3   |Grand Theft Auto IV                    |1268            |37679            |
|Switch         |The Legend of Zelda: Breath of the Wild|1122            |19891            |
|XboxOne        |Red Dead Redemption 2                  |1118            |17818            |
|Xbox           |Halo: Combat Evolved                   |793          

Print the name of the Platform with the highest percentage of titles of "Open-World" genre.

*Hint*: every game has multiple genres, so use ```pyspark.sql.Column.contains``` to impose the condition that the list of genres contains "Open-World".

In [ ]:
# CodeGrade Tag Ex4
### Print the name of the Platform with the highest percentage of titles of
### "Open-World" genre

open_world = F.when(F.col('genre').contains('Open-World'),F.countDistinct('name')).otherwise(0).alias('open_world_titles')

plat_OpenWorld = GamesDF.groupBy(['platform','genre']).agg(open_world,F.countDistinct('name').alias('all_titles')).groupBy('platform').agg((F.sum('open_world_titles')/F.sum('all_titles')).alias('portion_open_world_titles')).sort(('portion_open_world_titles'),ascending=(False)).first()[0]

print(f"Platform with the highest percentage of Open-World titles: {plat_OpenWorld}")

Platform with the highest percentage of Open-World titles: XboxSeriesX


⚠️ Let's define a title *Single Player* if and only if its feature ```player``` is equal either to "1 Player" or "No Online Multiplayer".

For example, *The Legend of Zelda: Ocarina of Time* has ```player``` equal "1 Player", so it is a Single Player. For another example, *Grim Fandango* has ```player``` equal "No Online Multiplayer", so it is a Single Player too. On the other hand, *Mario Kart 8* has ```player``` equal "Up to 12", so it is not a Single Player.

Havind defined Single Player titles as above,

* Print the total number of distinct Single Player titles.

* Print the total number of developers which have developed at least one Single Player title.

In [ ]:
# CodeGrade Tag Ex5
### Print the total number of distinct Single Player titles.
### Print the total number of developers which have developed at least one Single Player title.

SinglePlayer_titles = GamesDF.where('players in ("1 Player","No Online Multiplayer")').select(F.countDistinct('name')).first()[0]

SinglePlayer_developers = GamesDF.where('players in ("1 Player","No Online Multiplayer")').select(F.countDistinct('developer')).first()[0]

print(f"Number of Single Player titles: {SinglePlayer_titles}")
print(f"Number of developers of Single Player titles: {SinglePlayer_developers}")

Number of Single Player titles: 4838
Number of developers of Single Player titles: 2509
